In [ ]:
from nmjmc import systems
from nmjmc import sampling
from nmjmc import map_to_reference
from tqdm import tqdm_notebook
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
potential_parameters = {
    "nsolvent": 36,
    "eps": 1.0,  # LJ prefactor
    "rm": 1.0,  # LJ particle size
    "dimer_slope": 0.0,  # dimer slope parameter
    "dimer_a": 25.0,  # dimer x2 parameter
    "dimer_b": 10.0,  # dimer x4 parameter
    "dimer_dmid": 1.5,  # dimer transition state distance
    "dimer_k": 20.0,  # dimer force constant
    "box_halfsize": 3.0,
    "box_k": 100.0,  # box repulsion force constant
    "grid_k": 0.0,  # restraint strength to particle grid (to avoid permutation)
}

In [ ]:
system = systems.RepulsiveParticles(params=potential_parameters)
references = np.load("../local_data/particles_reference_configurations.npz")
reference_open = references["reference_open"]
reference_closed = references["reference_closed"]

In [ ]:
class BiasedParticleSystem:
    def __init__(
        self, system, reference_open, reference_closed, k_bias=5, dimer_split=1.5
    ):
        self.system = system
        self.reference_open = reference_open
        self.reference_closed = reference_closed
        self.k_bias = k_bias
        self.dimer_split = dimer_split

    def energy_bias(self, x):
        split = np.linalg.norm(x[:, :2] - x[:, 2:4], axis=1)
        idcs_open = np.where(split > self.dimer_split)[0]
        idcs_closed = np.where(split < self.dimer_split)[0]
        E_bias = np.zeros(len(x))
        E_bias[idcs_open] = np.sum((x[idcs_open] - self.reference_open) ** 2, axis=1)
        E_bias[idcs_closed] = np.sum(
            (x[idcs_closed] - self.reference_closed) ** 2, axis=1
        )
        return self.k_bias * E_bias

    def energy(self, x):
        E_system = self.system.energy(x)
        E_bias = self.energy_bias(x)
        return E_system + E_bias

In [ ]:
bias_k = np.array([1000, 10, 5, 2, 1, 0])
n_steps = 10_000_000
stride = 100

In [ ]:
samples_open = []
samples_closed = []
for bias in tqdm_notebook(bias_k):
    biased_system = BiasedParticleSystem(
        system, reference_open, reference_closed, k_bias=bias
    )
    sampler_closed = sampling.MCSampler(
        biased_system, reference_closed, 76, stride=stride
    )
    sampler_open = sampling.MCSampler(biased_system, reference_open, 76, stride=stride)
    sampler_open.run(n_steps, reporter="notebook")
    sampler_closed.run(n_steps, reporter="notebook")
    samples_open.append(map_to_reference(sampler_open.traj, reference_open))
    samples_closed.append(map_to_reference(sampler_closed.traj, reference_closed))
samples_open = np.array(samples_open)
samples_closed = np.array(samples_closed)

In [ ]:
np.savez(
    "../local_data/training_data_particles_biased",
    params=potential_parameters,
    samples_open=samples_open,
    samples_closed=samples_closed,
    bias_k=bias_k,
)